## Week 1
### Data Capture from Binance


In [1]:
pip install websockets pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import asyncio
import websockets
import json
import pandas as pd
import datetime
import os

In [38]:
buffer = []

async def binanceDepthCollector():
    dep_url = f"wss://stream.binance.com:9443/ws/btcusdt@depth10@100ms"
    async with websockets.connect(dep_url) as websocket:
        print('Connected')
        nexttime = (datetime.datetime.now()+datetime.timedelta(minutes=1)).replace(minute=0, second=0, microsecond=0)
        while True:
            try:
                info = await websocket.recv()
                #print('start listenning') #测一下
                data = json.loads(info)
                now = datetime.datetime.now()
                timestamp = now.strftime('%Y-%m-%d %H:%M:%S.%f')

                rows = []#存储当前访问的depth

                #print(data)#测一下读取到的数据格式

                for i, (bid,ask) in enumerate(zip(data['bids'],data['asks'])):
                    price1, qty1 = bid
                    price2, qty2 = ask
                    rows.append([timestamp,data['lastUpdateId'],[float(price1),float(qty1)],[float(price2),float(qty2)],i+1])

                buffer.extend(rows)#准备该小时写入的数据

                #我想的是每分钟保存一次csv数据，后面可以看情况并表，或者调参即可
                if now>=nexttime:
                    df = pd.DataFrame(buffer, columns = ['timestamp','lastUpdateId','bids','asks','level'])
                    filename_time = nexttime.strftime('%Y-%m-%d_%H-%M')
                    filename = f"depth10_btcusdt_{filename_time}.csv"
                    df.to_csv(filename, index = False)
                    print('fill saved')

                    buffer.clear()#清空缓存
                    nexttime = nexttime + datetime.timedelta(minutes=1)

            except Exception as e:
                print(f"[Error] {e}")
                await asyncio.sleep(5)  

In [39]:
await binanceDepthCollector()

Connected
start listenning
{'lastUpdateId': 72374036971, 'bids': [['108708.63000000', '0.99732000'], ['108708.62000000', '0.30461000'], ['108708.00000000', '0.02260000'], ['108707.54000000', '0.00137000'], ['108707.46000000', '0.00010000'], ['108707.45000000', '0.30020000'], ['108706.98000000', '0.00010000'], ['108706.97000000', '0.08015000'], ['108706.81000000', '0.00010000'], ['108706.80000000', '0.07711000']], 'asks': [['108708.64000000', '2.91863000'], ['108708.65000000', '0.00106000'], ['108708.67000000', '0.00039000'], ['108709.00000000', '0.00100000'], ['108709.02000000', '0.00010000'], ['108709.03000000', '0.18481000'], ['108709.04000000', '0.34066000'], ['108709.29000000', '0.00137000'], ['108709.48000000', '0.00010000'], ['108709.49000000', '0.08984000']]}
fill saved
start listenning
{'lastUpdateId': 72374037016, 'bids': [['108708.63000000', '0.54891000'], ['108708.62000000', '0.30461000'], ['108708.00000000', '0.02260000'], ['108707.54000000', '0.00137000'], ['108707.4600000

CancelledError: 

In [24]:
await websocket.close()

AttributeError: module 'websocket' has no attribute 'close'